The goal of this notebook is to perform a **model explainability analysis** for our **text classification model** using [Lime](https://github.com/marcotcr/lime). We will be using the dataset, processing steps, and baseline model (logistic regression) described in our [previous notebook](https://github.com/nickersonj/glg-capstone/blob/main/modeling/baseline/textclass_baseline_allthenews.ipynb).

## Setup

In [11]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.pipeline import make_pipeline
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

## Data Processing

In [2]:
# Load in the dataset
df = pd.read_csv('../modeling/baseline/data/all-the-news-25k.csv', low_memory=False)
df.head()

,date,year,month,day,title,article,section,publication
0,2018-05-02 17:09:00,2018,5.0,2,You Can Trick Your Brain Into Being More Focused,If only every day could be like this. You can’...,healthcare,Vice
1,2019-06-23 00:00:00,2019,6.0,23,Hudson's Bay's chairman's buyout bid pits reta...,(Reuters) - The success of Hudson’s Bay Co Exe...,business,Reuters
2,2018-12-28 00:00:00,2018,12.0,28,Wells Fargo to pay $575 million in settlement ...,NEW YORK (Reuters) - Wells Fargo & Co (WFC.N) ...,business,Reuters
3,2019-05-21 00:00:00,2019,5.0,21,Factbox: Investments by automakers in the U.S....,(Reuters) - Major automakers have announced a ...,business,Reuters
4,2019-02-05 00:00:00,2019,2.0,5,Exclusive: Britain's financial heartland unbow...,LONDON (Reuters) - Britain’s financial service...,business,Reuters


In [7]:
# Get a list of class names, whose order corresponds to their quantitative values used in modeling
class_names = df['section'].unique()
class_names

array(['healthcare', 'business', 'sports', 'technology', 'movies'],
      dtype=object)

In [4]:
# Convert the section column into numerical values
df['sectionId'] = df['section'].factorize()[0]
df.head()

,date,year,month,day,title,article,section,publication,sectionId
0,2018-05-02 17:09:00,2018,5.0,2,You Can Trick Your Brain Into Being More Focused,If only every day could be like this. You can’...,healthcare,Vice,0
1,2019-06-23 00:00:00,2019,6.0,23,Hudson's Bay's chairman's buyout bid pits reta...,(Reuters) - The success of Hudson’s Bay Co Exe...,business,Reuters,1
2,2018-12-28 00:00:00,2018,12.0,28,Wells Fargo to pay $575 million in settlement ...,NEW YORK (Reuters) - Wells Fargo & Co (WFC.N) ...,business,Reuters,1
3,2019-05-21 00:00:00,2019,5.0,21,Factbox: Investments by automakers in the U.S....,(Reuters) - Major automakers have announced a ...,business,Reuters,1
4,2019-02-05 00:00:00,2019,2.0,5,Exclusive: Britain's financial heartland unbow...,LONDON (Reuters) - Britain’s financial service...,business,Reuters,1


In [8]:
# Clean the article text
stop = stopwords.words('english')
df['clean_article'] = df['article'].map(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
def process_text(text):
    text = str(text).lower()
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", " ", text
    )
    text = " ".join(text.split())
    return text
df['text'] = df['clean_article'].apply(process_text)
df.head()

,date,year,month,day,title,article,section,publication,sectionId,clean_article,text
0,2018-05-02 17:09:00,2018,5.0,2,You Can Trick Your Brain Into Being More Focused,If only every day could be like this. You can’...,healthcare,Vice,0,If every day could like this. You can’t put fi...,if every day could like this you can’t put fin...
1,2019-06-23 00:00:00,2019,6.0,23,Hudson's Bay's chairman's buyout bid pits reta...,(Reuters) - The success of Hudson’s Bay Co Exe...,business,Reuters,1,(Reuters) - The success Hudson’s Bay Co Execut...,reuters the success hudson’s bay co executive ...
2,2018-12-28 00:00:00,2018,12.0,28,Wells Fargo to pay $575 million in settlement ...,NEW YORK (Reuters) - Wells Fargo & Co (WFC.N) ...,business,Reuters,1,NEW YORK (Reuters) - Wells Fargo & Co (WFC.N) ...,new york reuters wells fargo co wfc n pay 575 ...
3,2019-05-21 00:00:00,2019,5.0,21,Factbox: Investments by automakers in the U.S....,(Reuters) - Major automakers have announced a ...,business,Reuters,1,(Reuters) - Major automakers announced slew in...,reuters major automakers announced slew invest...
4,2019-02-05 00:00:00,2019,2.0,5,Exclusive: Britain's financial heartland unbow...,LONDON (Reuters) - Britain’s financial service...,business,Reuters,1,LONDON (Reuters) - Britain’s financial service...,london reuters britain’s financial services in...


## Logistic Regression Model

In [42]:
# Split the data into training and test sets
target = df['sectionId']
df_train, df_test = train_test_split(df, test_size=0.20, stratify=target, random_state=42)
X_train = df_train['text']
X_test = df_test['text'].values
y_train = df_train['sectionId']
y_test = df_test['sectionId'].values

# Convert to token counts, use a logistic regression model, and make a pipeline
vec = CountVectorizer(max_features=10000)
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
pipe = make_pipeline(vec, lr)

In [43]:
# Fit the model, predict on the test set, and print evaluation metrics
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=df['section'].unique()))
print("F1 score: ", f1_score(y_test, y_pred, average='weighted'))

/usr/local/Caskroom/miniconda/base/envs/glgenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:  0.9512
              precision    recall  f1-score   support

  healthcare       0.98      0.96      0.97      1000
    business       0.91      0.92      0.92      1000
      sports       0.97      0.98      0.98      1000
  technology       0.90      0.90      0.90      1000
      movies       0.99      0.99      0.99      1000

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000

F1 score:  0.9512473044656483


As expected, this F1 score reproduces the baseline model performance in our [previous notebook](https://github.com/nickersonj/glg-capstone/blob/main/modeling/baseline/textclass_baseline_allthenews.ipynb).

## Explainability with Lime

In [48]:
# Generate Lime explanations (with up to 6 features for all 5 labels) for a specific article
explainer = LimeTextExplainer(class_names=class_names)
idx = 0
exp = explainer.explain_instance(X_test[idx], pipe.predict_proba, num_features=6, labels=[0, 1, 2, 3, 4])
print('Article number: %d' % idx)
print('Predicted class =', class_names[y_pred[idx]])
print('True class: %s' % class_names[y_test[idx]])
print('\nExplanation for class %s' % class_names[0])
print('\n'.join(map(str, exp.as_list(label=0))))
print('\nExplanation for class %s' % class_names[1])
print('\n'.join(map(str, exp.as_list(label=1))))
print('\nExplanation for class %s' % class_names[2])
print('\n'.join(map(str, exp.as_list(label=2))))
print('\nExplanation for class %s' % class_names[3])
print('\n'.join(map(str, exp.as_list(label=3))))
print('\nExplanation for class %s' % class_names[4])
print('\n'.join(map(str, exp.as_list(label=4))))

Article number: 0
Predicted class = sports
True class: sports

Explanation for class healthcare
('fans', -0.0035552483804507425)
('nba', -0.0032349919446120613)
('players', -0.0031577121130896978)
('star', -0.0031203910133790784)
('team', -0.0028148298851396547)
('it', -0.002480644884938239)

Explanation for class business
('that', -0.00025532859967940165)
('players', -0.0002475703860147425)
('fans', -0.00023171504498669225)
('team', -0.00022325742618811695)
('game', -0.0002209134829888044)
('nba', -0.00021792417372510677)

Explanation for class sports
('nba', 0.03732558731674779)
('players', 0.03611918454403729)
('team', 0.026609167501998758)
('he', 0.023990141371108945)
('that', 0.021756679595989814)
('right', 0.02173864266644011)

Explanation for class technology
('he', -0.008953452031318997)
('all', -0.008770412964524335)
('nba', -0.008192474896822486)
('star', -0.007346869016267969)
('fans', -0.00718755594728779)
('team', -0.006847976002616628)

Explanation for class movies
('play

### Interpretation of Lime Output

The above explainability output shows the predicted class and true class for a particular article number, as well as the explanation for each class. Note that **positive and negative signs** denote words that are positive or negative for a particular class, so it is possible that a particular word that is negative towards one class may be positive towards another class.

For **healthcare**, we can see that its features are sports-related words (e.g., 'fans', 'nba', 'players', 'team') and that they are negatively inclined for the healthcare topic. It appears that the model is identifying healthcare articles by its lack of sports terms, as opposed to learning healthcare-related terminology. Some of these sports-related words (e.g., 'nba', 'players', 'team') show up as positive features for **sports**, as expected.

The same is the case for **business**, **technology**, and **movies**, where we again see that sports-related terms such as 'players', 'fans', 'team', 'game', and 'nba' are strong negative features. One exception is that it appears that the model has learned that the term 'star' is positively associated with **movies**, which makes sense.

To further improve our model, we may want to train with **additional articles for healthcare, business, technology, and movies** so that the model can better learn those classes. It would also be interesting to see how the model performs if sports articles are omitted completely; would the model still be able to differentiate between the remaining classes, given that sports-related terms are its top features?